# Santiago de Chile nighborhood analysis

## Introduction

When the COVID-19 pandemic ends, a lot of people will feel the rush to go outside, use parks, eat at restaurants, check museums, go to bars with friends and meet people at coffee shops. The thing is, the most popular zones will experience a big influx of people, so getting a table in your favorite bar will be difficult on a Saturday night, or maybe you just want to try something new.

Santiago is the biggest city of Chile, there are many places where you could go, you just  don't know them. If you can't go to Providencia, one the best places to go get a drink or grab a bite, you could search for similar neighborhoods to that one, so you can fulfill your plans and try something new, and trying new things is always exiting.

The scope of this project is the following: for each neighborhood in Santiago de Chile, check the most common type of places of interest, rank them by occurence and place a marker on a map with that information. The next step is clustering the different neighborhoods into similar ones based on the type of the places of interest, that way you can choose a substitute based on what you want.

You like Neighborhood X because it has a lot of places to eat and cultural landmarks, but you don't want to go again to the same places you are used to? No problem, check what neighborhoods are similar and give them a try!

## The Data

First, we need the Postal Codes of the neighborhoods of Santiago and we will scrap them from https://es.wikipedia.org/wiki/Anexo:C%C3%B3digos_postales_de_Chile, there is a table with every Postal Code of every neighborhood in Chile, so we will need to filter after the scrapping. Postal Codes between 7000000 and 8999999 correspond to nighborhoods inside the Metropolitan Area of Santiago, so we will use those.

Using geolocator library, we will get the Latitude and Longitude for every neighborhood and add them to the dataset.

With the Foursquare API, the venues of interest will be requested and added to a new dataframe, where they will be one hot encoded, grouped by neighborhood and standardized, so a clustering algorithm may be applied.